In [1]:
import pandas as pd
from azureml.core import Workspace, Experiment, Run
from azureml.train.automl import AutoMLConfig

In [2]:
storage_account_name = "<Storage account name>"
storage_account_key = "<Storage account key>"
container = "<Container name>"

In [3]:
spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

In [4]:
data = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ",") \
  .csv("wasbs://{0}@{1}.blob.core.windows.net/housing.csv".format(container, storage_account_name))

data.show()

In [5]:
df = data.toPandas()

In [6]:
df = pd.get_dummies(df, columns=["ocean_proximity"])

In [7]:
df = df.dropna(axis="rows")

In [8]:
ws = Workspace.from_config("/dbfs/FileStore", _file_name="config.json")

In [9]:
settings = {
    "iteration_timeout_minutes": 2,
    "iterations": 20,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "n_cross_validations": 5
}

In [10]:
automl_config = AutoMLConfig(
    task='regression',
    iteration_timeout_minutes=1,
    iterations=20,
    primary_metric='r2_score',
    training_data=df,
    label_column_name="median_house_value",
    n_cross_validations=5)

In [11]:
experiment = Experiment(ws, "house-prices")

In [12]:
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_98c4900f-65e9-496a-a00a-c4ba21947c10
Current status: DatasetCrossValidationSplit. Generating CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION PIPELINE DURATION METRIC BEST
 0 StandardScalerWrapper LightGBM 0:00:25 0.8419 0.8419
 1 MaxAbsScaler LightGBM 0:00:27 0.8313 0.8419
 2 StandardScalerWrapper ExtremeRandomTrees 0:00:18 0.4225 0.8419
 3 RobustScaler LightGBM 0:00:29 0.8333 0.8419
 4 StandardScalerWrapper LightGBM 0:00:20 0.7132 0.8419
 5 [20:05:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:05:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:05:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:05:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:05:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:05:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
StandardScalerWrapper XGBoostRegressor 0:00:15 0.7743 0.8419
 6 StandardScalerWrapper LightGBM 0:00:16 0.8408 0.8419
 7 StandardScalerWrapper ElasticNet 0:00:37 0.6450 0.8419
 8 StandardScalerWrapper ExtremeRandomTrees 0:00:20 0.6200 0.8419
 9 RobustScaler ElasticNet 0:00:20 0.6433 0.8419
 10 StandardScalerWrapper LassoLars 0:00:13 0.6450 0.8419
 11 MaxAbsScaler RandomForest 0:00:15 0.6271 0.8419
 12 MinMaxScaler ExtremeRandomTrees 0:00:14 0.6688 0.8419
 13 MinMaxScaler GradientBoosting 0:00:27 0.7976 0.8419
 14 SparseNormalizer LightGBM 0:00:17 0.6998 0.8419
 15 StandardScalerWrapper LightGBM 0:00:18 0.8420 0.8420
 16 StandardScalerWrapper ElasticNet 0:00:21 0.6173 0.8420
 17 MinMaxScaler DecisionTree 0:00:13 0.5847 0.8420
 18 VotingEnsemble 0:00:30 0.8485 0.8485
 19 StackEnsemble 0:00:35 0.8484 0.8485